In [ ]:
pip install pennylane

# GP

In [ ]:
################################################################################
# Quantum-Kernel-based BO for a 3-Qubit PennyLane Circuit Minimizing Ising Energy
################################################################################

import numpy as np
import math
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.special import erf

import pennylane as qml
from pennylane import numpy as pnp

##############################################################################
# 1) The Provided GaussianProcessRegressor Class (from your prompt)
##############################################################################
class GaussianProcessRegressor:
    """
    Gaussian Process Regressor for d-dimensional inputs.
    Returns both predictive mean and std if requested.
    """
    def __init__(self, kernel, alpha=1e-5):
        self.kernel = kernel
        self.alpha = alpha
        self.X_train = None
        self.y_train = None
        self.K_inv = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.X_train = X
        self.y_train = y
        K = self.kernel(X, X)
        K += self.alpha * np.eye(len(X))
        self.K_inv = np.linalg.inv(K)

    def predict(self, X_test, return_std=False):
        X_test = np.array(X_test)
        K_star = self.kernel(X_test, self.X_train)
        y_mean = K_star @ (self.K_inv @ self.y_train)

        if return_std:
            K_star_star = self.kernel(X_test, X_test)
            cov = K_star_star - K_star @ self.K_inv @ K_star.T
            var = np.diag(cov)
            var = np.maximum(var, 0.0)
            y_std = np.sqrt(var)
            return y_mean, y_std
        else:
            return y_mean

##############################################################################
# 2) Define the 3-qubit circuit & Hamiltonian for Ising-like model
##############################################################################
num_qubits = 3
num_layers = 5
dev = qml.device("default.qubit", wires=num_qubits, shots=None)

# We'll define an Ising Hamiltonian with effectively + X_jX_{j+1} + Z_j
# by setting Jx=-1, hz=-1 in the "paper's sign" => sum_j X_jX_j+1 + sum_j Z_j
# We'll just build a PennyLane Hamiltonian similarly:
obs = []
coeffs = []

# Coupling: X_j X_{j+1}
for j in range(num_qubits - 1):
    obs.append(qml.PauliX(j) @ qml.PauliX(j+1))
    coeffs.append(1.0)  # effectively +1

# Local Z
for j in range(num_qubits):
    obs.append(qml.PauliZ(j))
    coeffs.append(1.0)

H = qml.Hamiltonian(coeffs, obs)

# dimension => (2 + 2*num_layers)*num_qubits => (2+2*3)*3 => 24
D = (2 + 2*num_layers)*num_qubits

@qml.qnode(dev)
def circuit_energy(params_flat):
    """
    PennyLane QNode that prepares the 3-qubit, 3-layer circuit (24 params),
    then measures the expectation of H.
    """
    params_reshaped = params_flat.reshape((2*(num_layers+1), num_qubits))  # shape (8,3)
    # 1) initial block
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)

    # 2) repeated layers
    idx=2
    for _layer in range(num_layers):
        # entangle
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        # single-qubit
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2

    return qml.expval(H)

def cost_fn(params_flat):
    return float(circuit_energy(params_flat))

##############################################################################
# 3) Build a "quantum fidelity kernel" => overlap of states
##############################################################################
# We'll define a QNode that returns state, then compute fidelity = |<psi(x)|psi(y)>|^2
# For efficiency, store states in a cache so we don't recalc them repeatedly.

@qml.qnode(dev)
def circuit_state(params_flat):
    """
    Returns the state vector (1D complex array) from the same circuit,
    so we can compute fidelity.
    """
    params_reshaped = params_flat.reshape((2*(num_layers+1), num_qubits))
    # same structure as above
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)
    idx=2
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2
    return qml.state()

# We'll store states in an LRU cache for fast repeated calls
@lru_cache(None)
def get_state(params_tuple):
    """
    params_tuple is a tuple of 24 floats. We'll convert to a pnp.array and run circuit_state.
    We store the result in a cache for speed.
    """
    arr = pnp.array(params_tuple, dtype=pnp.float64)
    return circuit_state(arr)

def quantum_fidelity_kernel(X, Y):
    """
    For each pair (x_i, y_j) in X, Y, compute fidelity = |<psi(x_i)|psi(y_j)>|^2
    X => NxD, Y => MxD
    We'll do NxM result.
    We'll store states in a dictionary to avoid repeated circuit calls.
    """
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    N, D_ = X.shape
    M, D2_ = Y.shape
    assert D_==D2_, "Dimension mismatch"
    K = np.zeros((N,M), dtype=float)
    for i in range(N):
        # convert X[i] to tuple
        xi_tuple = tuple(X[i])
        psi_i = get_state(xi_tuple)  # 2^3=8 dim complex array
        # we only do conj once
        conj_psi_i = np.conjugate(psi_i)
        for j in range(M):
            yj_tuple = tuple(Y[j])
            psi_j = get_state(yj_tuple)
            overlap = np.vdot(psi_i, psi_j)  # or conj_psi_i @ psi_j
            # fidelity
            val = abs(overlap)**2
            K[i,j] = val
    return K

##############################################################################
# 4) Simple BO approach with this quantum kernel GP
##############################################################################
def expected_improvement(X_star, f_best, gp):
    """
    Standard EI => for each x in X_star, compute:
      mu, std => z=(f_best - mu)/std
      pdf, cdf => improvement = (f_best - mu)*cdf + std*pdf
    We pick the argmax.
    """
    mu, std = gp.predict(X_star, return_std=True)
    ei = np.zeros_like(mu)
    mask = std>1e-12
    z = (f_best - mu[mask]) / std[mask]
    pdf = 1./np.sqrt(2.*math.pi)*np.exp(-0.5*z**2)
    cdf = 0.5*(1.+erf(z/np.sqrt(2.)))
    improvement = (f_best - mu[mask])*cdf + std[mask]*pdf
    improvement = np.maximum(improvement, 0.)
    ei[mask] = improvement

    ei = -(mu - 2*std)
    return ei

def quantum_kernel_bo(n_init=5, n_iter=20, seed=0, n_cand=1000):
    np.random.seed(seed)
    # dimension is 24
    # 1) gather initial data
    X_data = np.random.rand(n_init, D)*2.*math.pi
    y_data = np.array([cost_fn(row) for row in X_data])

    # 2) build GP with quantum_fidelity_kernel
    gp = GaussianProcessRegressor(kernel=quantum_fidelity_kernel, alpha=1e-9)
    gp.fit(X_data, y_data)

    best_history = [y_data.min()]

    for step in range(n_iter):
        # 3) find next candidate x via EI over random set
        f_best = y_data.min()
        X_cand = np.random.rand(n_cand, D)*2.*math.pi
        ei_vals = expected_improvement(X_cand, f_best, gp)
        idx_best = np.argmax(ei_vals)
        x_next = X_cand[idx_best]
        y_next = cost_fn(x_next)
        # 4) update training data
        X_data = np.vstack([X_data, x_next])
        y_data = np.concatenate([y_data, [y_next]])
        gp.fit(X_data, y_data)
        best_so_far = min(best_history[-1], y_next)
        best_history.append(best_so_far)

        print(f"Iteration {step+1:2d}, best so far = {best_so_far:.6f}")

    return X_data, y_data, best_history

##############################################################################
# 5) Run it
##############################################################################
if __name__=="__main__":
    X_data, y_data, best_hist = quantum_kernel_bo(n_init=5, n_iter=100, seed=None, n_cand=100)
    print("Final best energy found:", best_hist[-1])
    plt.plot(best_hist, label="Best energy so far")
    plt.xlabel("Iteration")
    plt.ylabel("Energy")
    plt.title("BO with Quantum Fidelity Kernel (3-Qubit Ising, 24 params)")
    plt.legend()
    plt.show()


## projected kernels

In [ ]:
##############################################################################
# 1) Imports & Setup
##############################################################################
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.special import erf

import pennylane as qml
from pennylane import numpy as pnp

##############################################################################
# 2) GaussianProcessRegressor (unchanged from your prompt)
##############################################################################
class GaussianProcessRegressor:
    """
    Gaussian Process Regressor for d-dimensional inputs.
    Returns both predictive mean and std if requested.
    """
    def __init__(self, kernel, alpha=1e-5):
        self.kernel = kernel
        self.alpha = alpha
        self.X_train = None
        self.y_train = None
        self.K_inv = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.X_train = X
        self.y_train = y
        K = self.kernel(X, X)
        K += self.alpha * np.eye(len(X))
        self.K_inv = np.linalg.inv(K)

    def predict(self, X_test, return_std=False):
        X_test = np.array(X_test)
        K_star = self.kernel(X_test, self.X_train)
        y_mean = K_star @ (self.K_inv @ self.y_train)

        if return_std:
            K_star_star = self.kernel(X_test, X_test)
            cov = K_star_star - K_star @ self.K_inv @ K_star.T
            var = np.diag(cov)
            var = np.maximum(var, 0.0)
            y_std = np.sqrt(var)
            return y_mean, y_std
        else:
            return y_mean

##############################################################################
# 3) Setup: 3-qubit circuit, Ising Hamiltonian, cost function
##############################################################################
num_qubits = 5
num_layers = 1
dev = qml.device("default.qubit", wires=num_qubits, shots=None)

# Build H = sum_j X_jX_{j+1} + sum_j Z_j
obs = []
coeffs = []
for j in range(num_qubits-1):
    obs.append(qml.PauliX(j) @ qml.PauliX(j+1))
    coeffs.append(1.0)
for j in range(num_qubits):
    obs.append(qml.PauliZ(j))
    coeffs.append(1.0)
H = qml.Hamiltonian(coeffs, obs)

# dimension => (2 + 2*num_layers)*num_qubits => 24 if num_layers=3,
# but here we set num_layers=1 for a simpler example => (2+2*1)*3=6 parameters
D = (2 + 2*num_layers)*num_qubits

@qml.qnode(dev)
def circuit_energy(params_flat):
    """
    Prepare the circuit (with 'num_layers'=1) and measure expectation of H.
    """
    params_reshaped = params_flat.reshape((2*(num_layers+1), num_qubits))
    # 1) initial block
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)
    idx=2
    # repeated layers
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2
    return qml.expval(H)

def cost_fn(params_flat):
    return float(circuit_energy(params_flat))

##############################################################################
# 4) Partial-Density QNode => measure subset of qubits' density matrix
##############################################################################
# We'll specify `subset_wires` as a list of qubit indices. Then we do
# qml.density_matrix(subset_wires). This returns a 2^(|subset|) x 2^(|subset|) complex matrix,
# which might be mixed if the global state is partially traced out.

@qml.qnode(dev)
def circuit_partial_density(params_flat, subset_wires):
    """
    Returns the partial density matrix for a specified subset of qubits.
    The rest are traced out automatically by PennyLane.
    """
    # same circuit as above
    params_reshaped = params_flat.reshape((2*(num_layers+1), num_qubits))
    # initial
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)
    idx=2
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2

    return qml.density_matrix(wires=subset_wires)

@lru_cache(None)
def get_partial_density(params_tuple, subset_wires_tuple):
    """
    Cache for partial density matrix.
    PennyLane expects a python list for `wires=...`,
    but we can't store a list as a cache key, so we use a tuple of int.
    """
    arr = pnp.array(params_tuple, dtype=pnp.float64)
    # convert subset_wires_tuple back to a list
    wires_list = list(subset_wires_tuple)
    dm = circuit_partial_density(arr, wires_list)
    return dm

def partial_density_kernel(X, Y, subset_wires=[0,1], method="hilbert_schmidt"):
    """
    Projected kernel on a subset of qubits.
    For each x_i,y_j, we get partial density matrices => dm_x, dm_y.
    Then define kernel = Tr(dm_x dm_y) if method="hilbert_schmidt"
    (One can define other overlaps, e.g. Uhlmann fidelity, but we do simpler HS.)
    """
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    N, dx = X.shape
    M, dx2 = Y.shape
    assert dx==dx2, "Dimension mismatch"

    # Turn subset_wires into a tuple so it can be used in a cache key
    wires_tuple = tuple(subset_wires)
    K = np.zeros((N,M), dtype=float)
    for i in range(N):
        x_tuple = tuple(X[i])
        dm_x = get_partial_density(x_tuple, wires_tuple)
        for j in range(M):
            y_tuple = tuple(Y[j])
            dm_y = get_partial_density(y_tuple, wires_tuple)
            if method=="hilbert_schmidt":
                # HS overlap = Tr(dm_x dm_y)
                # dm_x, dm_y are pnp arrays => convert to np
                val = np.real(np.trace(dm_x @ dm_y))
                K[i,j]=val
            else:
                raise ValueError("Only 'hilbert_schmidt' method is implemented.")
    return K

##############################################################################
# 5) Basic Bayesian Optimization with random candidate sampling + EI
##############################################################################
def expected_improvement(X_star, f_best, gp):
    mu, std = gp.predict(X_star, return_std=True)
    ei = np.zeros_like(mu)
    mask = std>1e-12
    z = (f_best - mu[mask]) / std[mask]
    pdf = 1./np.sqrt(2.*math.pi)*np.exp(-0.5*z**2)
    cdf = 0.5*(1.+erf(z/np.sqrt(2.)))
    improvement = (f_best - mu[mask])*cdf + std[mask]*pdf
    improvement = np.maximum(improvement, 0.)
    ei[mask]=improvement
    return ei

def bo_with_partial_kernel(
    n_init=5,
    n_iter=20,
    n_cand=100,
    seed=0,
    dim = 7
):
    np.random.seed(seed)
    # init data
    X_data = np.random.rand(n_init, D)*2.*math.pi
    y_data = np.array([cost_fn(x) for x in X_data])
    # build kernel
    def kernel_fn1(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[0], method="hilbert_schmidt")
    def kernel_fn2(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[1], method="hilbert_schmidt")
    def kernel_fn3(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[2], method="hilbert_schmidt")
    def kernel_fn4(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[0,2], method="hilbert_schmidt")
    def kernel_fn5(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[1,2], method="hilbert_schmidt")
    def kernel_fn6(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[0,1], method="hilbert_schmidt")
    def kernel_fn7(X, Y):
        return partial_density_kernel(X, Y, subset_wires=[0,1,2], method="hilbert_schmidt")

    kernels = [kernel_fn1,kernel_fn2,kernel_fn3,kernel_fn4,kernel_fn5,kernel_fn6]

    def kernel_fn(X, Y):
        print(dim)
        if dim == 7:
          return kernel_fn7(X, Y)
        else:
          res = 0
          for i in range(dim):
            res += kernels[i](X,Y)
          return res

    gp = GaussianProcessRegressor(kernel=kernel_fn, alpha=1e-8)
    gp.fit(X_data, y_data)

    best_hist=[y_data.min()]
    for step in range(n_iter):
        f_best = y_data.min()
        X_cand = np.random.rand(n_cand, D)*2.*math.pi
        ei_vals=expected_improvement(X_cand, f_best, gp)
        idx=np.argmax(ei_vals)
        x_next=X_cand[idx]
        y_next=cost_fn(x_next)

        X_data = np.vstack([X_data, x_next])
        y_data = np.concatenate([y_data, [y_next]])
        gp.fit(X_data, y_data)
        best_so_far=min(best_hist[-1], y_next)
        best_hist.append(best_so_far)
        #print(f"Iteration {step+1:2d}, best so far = {best_so_far:.6f}")
    return X_data, y_data, best_hist

##############################################################################
# 6) Run
##############################################################################
# if __name__=="__main__":
#     # Example usage: define partial kernel on qubits [0,2]
#     # Then do BO

#     trials = 3
#     energy_final = []
#     energy_final_std = []

#     for dim in range(1,8):
#         energies = []
#         for trial in range(trials):
#             X_data, y_data, best_hist = bo_with_partial_kernel(
#                 n_init=3,
#                 n_iter=100,
#                 n_cand=100,
#                 seed=None,
#                 dim = dim
#             )
#             energies.append(best_hist)
#             print(f"dim {dim} completed trial {trial}")
#             print("Final best energy found:", best_hist[-1])
#         energy_all = np.array(energies)
#         avg_energy = energy_all.mean(axis=0)
#         std_energy = energy_all.std(axis=0)

#         energy_final.append(avg_energy[-1])
#         energy_final_std.append(std_energy[-1])



#     plt.errorbar(np.array([1,2,3,4,5,6,7]), np.array(energy_final), np.array(energy_final_std), ls="-",
#                 marker='d',
#                 color="#009E73",
#                 alpha=1.0,
#                 capsize=4)


# GP with LFGBS

In [ ]:
##############################################################################
# Imports
##############################################################################
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.optimize import minimize
from scipy.special import erf

import pennylane as qml
from pennylane import numpy as pnp

##############################################################################
# 1) The same GaussianProcessRegressor class from your prompt
##############################################################################
class GaussianProcessRegressor:
    """
    Gaussian Process Regressor for d-dimensional inputs.
    Returns both predictive mean and std if requested.
    """
    def __init__(self, kernel, alpha=1e-5):
        self.kernel = kernel
        self.alpha = alpha
        self.X_train = None
        self.y_train = None
        self.K_inv = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.X_train = X
        self.y_train = y
        K = self.kernel(X, X)
        K += self.alpha * np.eye(len(X))
        self.K_inv = np.linalg.inv(K)

    def predict(self, X_test, return_std=False):
        X_test = np.array(X_test)
        K_star = self.kernel(X_test, self.X_train)
        y_mean = K_star @ (self.K_inv @ self.y_train)

        if return_std:
            K_star_star = self.kernel(X_test, X_test)
            cov = K_star_star - K_star @ self.K_inv @ K_star.T
            var = np.diag(cov)
            var = np.maximum(var, 0.0)
            y_std = np.sqrt(var)
            return y_mean, y_std
        else:
            return y_mean

##############################################################################
# 2) Define 3-qubit circuit (24 params) + cost function
##############################################################################
num_qubits = 3
num_layers = 1
D = (2 + 2*num_layers)*num_qubits  # 24
dev = qml.device("default.qubit", wires=num_qubits, shots=None)

obs = []
coeffs = []
# Coupling X_j X_{j+1}
for j in range(num_qubits - 1):
    obs.append(qml.PauliX(j) @ qml.PauliX(j+1))
    coeffs.append(1.0)
# Local Z
for j in range(num_qubits):
    obs.append(qml.PauliZ(j))
    coeffs.append(1.0)

H = qml.Hamiltonian(coeffs, obs)

@qml.qnode(dev)
def circuit_energy(params):
    params_reshaped = params.reshape((2*(num_layers+1), num_qubits))
    # initial block
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)
    idx=2
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2
    return qml.expval(H)

def cost_fn(params):
    return float(circuit_energy(params))

##############################################################################
# 3) Quantum Fidelity Kernel (Same as previous "fidelity" approach)
##############################################################################
@qml.qnode(dev)
def circuit_state(params):
    """
    Return state vector from same circuit. We'll cache it for fidelity calcs.
    """
    params_reshaped = params.reshape((2*(num_layers+1), num_qubits))
    # same arrangement
    for q in range(num_qubits):
        qml.RY(params_reshaped[0,q], wires=q)
        qml.RZ(params_reshaped[1,q], wires=q)
    idx=2
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx,q], wires=q)
            qml.RZ(params_reshaped[idx+1,q], wires=q)
        idx+=2
    return qml.state()

@lru_cache(None)
def get_cached_state(params_tuple):
    arr = pnp.array(params_tuple, dtype=pnp.float64)
    return circuit_state(arr)

def quantum_fidelity_kernel(X, Y):
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    N, D_ = X.shape
    M, D2_ = Y.shape
    assert D_==D2_, "Dimension mismatch"
    K = np.zeros((N,M), dtype=float)
    for i in range(N):
        xi_tuple = tuple(X[i])
        psi_i = get_cached_state(xi_tuple)
        for j in range(M):
            yj_tuple = tuple(Y[j])
            psi_j = get_cached_state(yj_tuple)
            overlap = np.vdot(psi_i, psi_j)
            val = abs(overlap)**2
            K[i,j]=val
    return K

##############################################################################
# 4) Implement a standard EI function (like in Bayesian Opt).
#    We'll do a naive negative-EI for LBFGS, so we'll define:
##############################################################################
def gp_ei(x, f_best, gp):
    """
    Return negative-EI for the point x, so we can do minimization => we want to
    minimize -EI => maximize EI
    """
    # x shape => (D,)
    x = x[None,:]  # shape(1,D)
    mu, std = gp.predict(x, return_std=True)
    mu, std = mu[0], std[0]
    eps=1e-12
    if std<eps:
        return 0.0  # or -0 => no improvement
    z=(f_best - mu)/std
    pdf=1./np.sqrt(2.*math.pi)*math.exp(-0.5*z**2)
    cdf=0.5*(1.+erf(z/np.sqrt(2.)))
    improvement = (f_best - mu)*cdf + std*pdf
    if improvement<0.0:
        improvement=0.0
    return -improvement  # negative => we'll do minimize

def gp_ei_grad(x, f_best, gp):
    """
    We'll do naive finite difference gradient for shape(D,).
    x => shape(D,)
    We'll do step=1e-4 or so => partial derivative of gp_ei w.r.t x[d].
    This is an example. It's possibly expensive for D=24 but demonstrates the approach.
    """
    grad = np.zeros_like(x)
    fx = gp_ei(x, f_best, gp)
    step=1e-4
    for d in range(len(x)):
        old_val = x[d]
        x[d] = old_val+step
        fplus = gp_ei(x, f_best, gp)
        x[d] = old_val
        grad[d] = (fplus - fx)/step
    return grad

##############################################################################
# 5) The iterative Bayesian Optimization approach with L-BFGS to refine candidate
##############################################################################
def bo_with_lbfgs(
    kernel_fn=quantum_fidelity_kernel,
    n_init=5,
    n_iter=15,
    seed=0,
    n_cand=1000
):
    np.random.seed(seed)
    # 1) gather initial data
    X_data = np.random.rand(n_init, D)*2.*math.pi
    y_data = np.array([cost_fn(row) for row in X_data])

    gp = GaussianProcessRegressor(kernel=kernel_fn, alpha=1e-9)
    gp.fit(X_data, y_data)

    best_hist=[y_data.min()]

    for step in range(n_iter):
        # f_best
        f_best = y_data.min()
        # 2) do small random search => pick best from 1000
        X_cand = np.random.rand(n_cand, D)*2.*math.pi
        ei_vals=[]
        for row in X_cand:
            ei_vals.append(-gp_ei(row, f_best, gp))  # we store negativeEI to see how large
        ei_vals=np.array(ei_vals)
        idx=np.argmax(ei_vals) # min of negative => max of EI
        x0 = X_cand[idx].copy()

        # 3) run LBFGS from x0 to refine
        def fun_and_grad(z):
            val = gp_ei(z, f_best, gp)
            g = gp_ei_grad(z, f_best, gp)
            return val, g
        bounds=[(0.,2.*math.pi)]*D
        res = minimize(fun_and_grad, x0, method='L-BFGS-B', jac=True, bounds=bounds,
                       options={'maxiter':30})
        x_next=res.x
        y_next=cost_fn(x_next)

        # 4) update GP
        X_data = np.vstack([X_data, x_next])
        y_data = np.concatenate([y_data, [y_next]])
        gp.fit(X_data, y_data)

        current_best=min(best_hist[-1], y_next)
        best_hist.append(current_best)
        print(f"Iteration {step+1:2d}, best so far = {current_best:.6f}, y_next={y_next:.6f}")

    return X_data, y_data, best_hist

##############################################################################
# 6) Run
##############################################################################
if __name__=="__main__":
    Xd, yd, bhist = bo_with_lbfgs(kernel_fn=quantum_fidelity_kernel,
                                  n_init=5,
                                  n_iter=200,
                                  seed=None,
                                  n_cand=200)
    print("Final best energy found:", bhist[-1])
    plt.plot(bhist, label="Best so far")
    plt.xlabel("Iteration")
    plt.ylabel("Energy")
    plt.title("BO with Quantum Fidelity Kernel + LBFGS Refinement")
    plt.legend()
    plt.show()


# GP LFBGS with different kernel, Final Experiment for running

In [ ]:
##############################################################################
# Imports
##############################################################################
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import lru_cache
from scipy.optimize import minimize
from scipy.special import erf

import pennylane as qml
from pennylane import numpy as pnp

##############################################################################
# 1) GaussianProcessRegressor (unchanged)
##############################################################################
class GaussianProcessRegressor:
    """
    Gaussian Process Regressor for d-dimensional inputs.
    Returns both predictive mean and std if requested.
    """
    def __init__(self, kernel, alpha=1e-5):
        self.kernel = kernel
        self.alpha = alpha
        self.X_train = None
        self.y_train = None
        self.K_inv = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.X_train = X
        self.y_train = y
        K = self.kernel(X, X)
        K += self.alpha * np.eye(len(X))
        self.K_inv = np.linalg.inv(K)

    def predict(self, X_test, return_std=False):
        X_test = np.array(X_test)
        K_star = self.kernel(X_test, self.X_train)
        y_mean = K_star @ (self.K_inv @ self.y_train)

        if return_std:
            K_star_star = self.kernel(X_test, X_test)
            cov = K_star_star - K_star @ self.K_inv @ K_star.T
            var = np.diag(cov)
            var = np.maximum(var, 0.0)
            y_std = np.sqrt(var)
            return y_mean, y_std
        else:
            return y_mean

##############################################################################
# 2) Define 3-qubit circuit (24 params) + cost function
#    (Use num_layers=3 => D=24)
##############################################################################
num_qubits = 3
num_layers = 1
D = (2 + 2*num_layers)*num_qubits  # => (2+2*3)*3=8*3=24
dev = qml.device("default.qubit", wires=num_qubits, shots=None)

##############################################################################
# Build the Ising Hamiltonian: sum_j X_j X_{j+1} + sum_j Z_j
##############################################################################
obs = []
coeffs = []
# Coupling X_j X_{j+1}
for j in range(num_qubits - 1):
    obs.append(qml.PauliX(j) @ qml.PauliX(j+1))
    coeffs.append(1.0)
# Local Z
for j in range(num_qubits):
    obs.append(qml.PauliZ(j))
    coeffs.append(1.0)

H = qml.Hamiltonian(coeffs, obs)

@qml.qnode(dev)
def circuit_energy(params):
    """
    Prepare a 3-qubit circuit with num_layers=3 => 24 parameters.
    Return expectation of H.
    """
    params_reshaped = params.reshape((2*(num_layers+1), num_qubits))
    # initial block
    for q in range(num_qubits):
        qml.RY(params_reshaped[0, q], wires=q)
        qml.RZ(params_reshaped[1, q], wires=q)

    idx = 2
    for _layer in range(num_layers):
        # Entangling
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[0, 2])
        qml.CNOT(wires=[1, 2])
        # Single-qubit rotations
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx, q], wires=q)
            qml.RZ(params_reshaped[idx+1, q], wires=q)
        idx += 2

    return qml.expval(H)

def cost_fn(params):
    return float(circuit_energy(params))

##############################################################################
# 3) Partial-density QNode + partial_density_kernel
##############################################################################
@qml.qnode(dev)
def circuit_partial_density(params, subset_wires):
    """
    Returns the partial density matrix for a specified subset of qubits.
    The rest are traced out automatically by PennyLane.
    """
    params_reshaped = params.reshape((2*(num_layers+1), num_qubits))
    # same circuit
    for q in range(num_qubits):
        qml.RY(params_reshaped[0, q], wires=q)
        qml.RZ(params_reshaped[1, q], wires=q)
    idx = 2
    for _layer in range(num_layers):
        qml.CNOT(wires=[0,1])
        qml.CNOT(wires=[0,2])
        qml.CNOT(wires=[1,2])
        for q in range(num_qubits):
            qml.RY(params_reshaped[idx, q], wires=q)
            qml.RZ(params_reshaped[idx+1, q], wires=q)
        idx += 2

    return qml.density_matrix(wires=subset_wires)

@lru_cache(None)
def get_partial_density(params_tuple, subset_wires_tuple):
    """
    Cached partial density matrix.
    """
    arr = pnp.array(params_tuple, dtype=pnp.float64)
    dm = circuit_partial_density(arr, list(subset_wires_tuple))
    return dm

def partial_density_kernel(X, Y, subset_wires=[0], method="hilbert_schmidt"):
    """
    Projected kernel on specified subset of qubits.
    For each x_i,y_j => partial density matrices => dm_x, dm_y => Tr(dm_x dm_y).
    """
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    N, dx = X.shape
    M, dx2 = Y.shape
    assert dx == dx2, "Dimension mismatch"

    wires_tuple = tuple(subset_wires)
    K = np.zeros((N, M), dtype=float)

    for i in range(N):
        x_tuple = tuple(X[i])
        dm_x = get_partial_density(x_tuple, wires_tuple)
        for j in range(M):
            y_tuple = tuple(Y[j])
            dm_y = get_partial_density(y_tuple, wires_tuple)
            # Hilbert-Schmidt => Tr(dm_x dm_y)
            val = np.real(np.trace(dm_x @ dm_y))
            K[i, j] = val
    return K

##############################################################################
# 4) We'll define multiple partial kernels for demonstration
##############################################################################
def kernel_fn1(X, Y):
    # single-qubit overlap on qubit 0
    return partial_density_kernel(X, Y, subset_wires=[0], method="hilbert_schmidt")

def kernel_fn2(X, Y):
    # sum of partial overlaps on qubits 0 and 1
    return (partial_density_kernel(X, Y, subset_wires=[0]) +
           partial_density_kernel(X, Y, subset_wires=[1]))

def kernel_fn3(X, Y):
    # sum of partial overlaps on qubits 0,1,2 separately
    return (partial_density_kernel(X, Y, subset_wires=[0]) +
            partial_density_kernel(X, Y, subset_wires=[1]) +
            partial_density_kernel(X, Y, subset_wires=[2]))

def kernel_fn4(X, Y):
    # partial overlap on subset [0,1]
    return partial_density_kernel(X, Y, subset_wires=[0,1])

def kernel_fn5(X, Y):
    return (partial_density_kernel(X, Y, subset_wires=[0,1]) +
            partial_density_kernel(X, Y, subset_wires=[1,2]))

def kernel_fn6(X, Y):
    return (partial_density_kernel(X, Y, subset_wires=[0,1]) +
            partial_density_kernel(X, Y, subset_wires=[1,2]) +
            partial_density_kernel(X, Y, subset_wires=[0,2]))

def kernel_fn7(X, Y):
    # full (no partial trace) => subset_wires=[0,1,2]
    return partial_density_kernel(X, Y, subset_wires=[0,1,2])

KERNELS = [kernel_fn1, kernel_fn2, kernel_fn3, kernel_fn4, kernel_fn5, kernel_fn6, kernel_fn7]

##############################################################################
# 5) Define EI + gradient for LBFGS (from second snippet, unchanged logic)
##############################################################################
def gp_ei(x, f_best, gp):
    """
    Return negative-EI for the point x => we do 'minimize' =>
    so negative-EI is what we minimize to find maximum of EI.
    """
    x = x[None, :]  # shape (1,D)
    mu, std = gp.predict(x, return_std=True)
    mu, std = mu[0], std[0]
    eps = 1e-12
    if std < eps:
        # No improvement if variance is tiny
        return 0.0
    z = (f_best - mu)/std
    pdf = 1./np.sqrt(2.*math.pi)*math.exp(-0.5*z**2)
    cdf = 0.5*(1.+erf(z/np.sqrt(2.)))
    improvement = (f_best - mu)*cdf + std*pdf
    if improvement < 0.0:
        improvement = 0.0
    return -improvement  # negative => we'll do minimize

def gp_ei_grad(x, f_best, gp):
    """
    Naive finite-difference gradient for gp_ei.
    x => shape(D,)
    """
    grad = np.zeros_like(x)
    fx = gp_ei(x, f_best, gp)
    step = 1e-4
    for d in range(len(x)):
        old_val = x[d]
        x[d] = old_val + step
        fplus = gp_ei(x, f_best, gp)
        x[d] = old_val
        grad[d] = (fplus - fx)/step
    return grad

##############################################################################
# 6) bo_with_lbfgs => Main iterative routine that uses partial kernel
##############################################################################
def bo_with_lbfgs(
    kernel_fn,
    n_init=5,
    n_iter=50,
    seed=0,
    n_cand=100
):
    """
    Perform Bayesian Optimization with a given kernel_fn,
    using L-BFGS to refine each new candidate.
    """
    np.random.seed(seed)
    # 1) gather initial data
    X_data = np.random.rand(n_init, D)*2.*math.pi
    y_data = np.array([cost_fn(row) for row in X_data])

    gp = GaussianProcessRegressor(kernel=kernel_fn, alpha=1e-9)
    gp.fit(X_data, y_data)

    best_hist = [y_data.min()]

    for step in range(n_iter):
        # current best
        f_best = y_data.min()
        # 2) do random search => pick best among n_cand
        X_cand = np.random.rand(n_cand, D)*2.*math.pi
        ei_vals = []
        for row in X_cand:
            # store negativeEI => we want largest positive EI
            ei_vals.append(-gp_ei(row, f_best, gp))
        ei_vals = np.array(ei_vals)
        idx = np.argmax(ei_vals)
        x0 = X_cand[idx].copy()

        # 3) refine with L-BFGS
        def fun_and_grad(z):
            val = gp_ei(z, f_best, gp)
            g = gp_ei_grad(z, f_best, gp)
            return val, g

        bounds = [(0., 2.*math.pi)]*D
        res = minimize(fun_and_grad, x0, method='L-BFGS-B',
                       jac=True, bounds=bounds,
                       options={'maxiter': 30})

        x_next = res.x
        y_next = cost_fn(x_next)

        # 4) update GP
        X_data = np.vstack([X_data, x_next])
        y_data = np.concatenate([y_data, [y_next]])
        gp.fit(X_data, y_data)

        current_best = min(best_hist[-1], y_next)
        best_hist.append(current_best)
        print(f"Iteration {step+1:2d}, best so far = {current_best:.6f}, y_next={y_next:.6f}")

    return X_data, y_data, best_hist




In [ ]:
import numpy as np
import math

def full_kernel_classic(x,y):

    #print(x,y)

    return kernel_fn7(np.array([x]),np.array([y]))[0][0]

def p_greedy_newton_flexible(kernel_func, Omega_b, max_m):
    """
    P-greedy on a discrete set Omega_b, up to max_m basis functions.

    Returns a dict pgreedy_info with:
      - 'm': number of basis chosen (= max_m, unless Omega_b empty)
      - 'center_pts': array of shape (m, d) for the chosen centers
      - 'chosen_indices': the indices in Omega_b for each chosen center
      - 'Nvals': shape (m, nB), Nvals[k, i] = N_{k+1}(Omega_b[i])
      - 'denominators': shape (m,), the normalization used at each step

    Using this info, we can evaluate the Newton basis for new points x.
    """
    nB = len(Omega_b)
    if nB == 0 or max_m == 0:
        return {
            'm': 0,
            'center_pts': np.zeros((0,0)),
            'chosen_indices': [],
            'Nvals': np.zeros((0,0)),
            'denominators': np.zeros(0)
        }

    # Precompute diagonal K(x_i,x_i)
    diagK = np.array([kernel_func(Omega_b[i],Omega_b[i]) for i in range(nB)])
    Nvals = np.zeros((max_m, nB))  # Nvals[k, i] => N_{k+1}(Omega_b[i])
    denominators = np.zeros(max_m)
    chosen_indices = []
    center_pts = []

    # 1) pick first center
    i1 = np.argmax(diagK)
    chosen_indices.append(i1)
    center_pts.append(Omega_b[i1])

    denom0 = math.sqrt(diagK[i1]) if diagK[i1]>1e-14 else 1e-14
    denominators[0] = denom0

    # define N_1(x_i)
    for i in range(nB):
        #print(Omega_b[i].shape, Omega_b[i1].shape)
        Nvals[0, i] = kernel_func(Omega_b[i], Omega_b[i1]) / denom0

    # power function squared
    P2 = diagK - Nvals[0,:]**2

    m=1
    while m<max_m:
        # print(diagK.shape)
        # print(Nvals.shape)
        # print(P2.shape)
        i_next = np.argmax(P2)
        chosen_indices.append(i_next)
        center_pts.append(Omega_b[i_next])

        denom = math.sqrt(P2[i_next]) if P2[i_next]>1e-14 else 1e-14
        denominators[m] = denom

        for i in range(nB):
            #print(Omega_b[i].shape, Omega_b[i_next].shape)
            val = kernel_func(Omega_b[i], Omega_b[i_next])
            # subtract sum_{r=0..m-1} Nvals[r, i_next]*Nvals[r, i]
            tmp=0.0
            for r in range(m):
                tmp += Nvals[r, i_next]*Nvals[r, i]
            val -= tmp
            Nvals[m, i] = val/denom

        P2 = P2 - Nvals[m,:]**2
        m+=1

    center_pts = np.array(center_pts)  # shape (m,d)

    pgreedy_info = {
        'm': m,
        'center_pts': center_pts,
        'chosen_indices': chosen_indices,
        'Nvals': Nvals,
        'denominators': denominators
    }
    return pgreedy_info

def evaluate_newton_basis(x, kernel_func, pgreedy_info, Omega_b):
    """
    For a new point x, compute [N_1(x), ..., N_m(x)] using the chosen centers from pgreedy_info.
    This replicates the recursion used in p_greedy_newton_flexible for the discrete set,
    but for x in continuous space.

    We need:
      - center_pts[k] = the chosen center for the (k+1)-th basis
      - denominators[k]
      - Nvals[r, i_k] = N_r(center_pts[k]) if i_k= chosen_indices[k]
    """
    m = pgreedy_info['m']
    if m==0:
        return np.zeros(0)
    center_pts = pgreedy_info['center_pts']
    chosen_indices = pgreedy_info['chosen_indices']
    Nvals = pgreedy_info['Nvals']
    denominators = pgreedy_info['denominators']

    newton_x = np.zeros(m)

    # 1) N_1(x)
    val = kernel_func(x, center_pts[0])
    newton_x[0] = val/denominators[0]

    # 2) subsequent
    for k in range(1,m):
        val = kernel_func(x, center_pts[k])

        # subtract sum_{r=0..k-1} [N_r(center_pts[k]) * N_r(x)]
        # but N_r(center_pts[k]) = Nvals[r, chosen_indices[k]]
        tmp = 0.0
        i_k = chosen_indices[k]
        for r in range(k):
            tmp += Nvals[r, i_k]*newton_x[r]

        val -= tmp
        newton_x[k] = val / denominators[k]

    return newton_x

def approximate_kernel_flexible(x, y, kernel_func, pgreedy_info, Omega_b):
    """
    approximate kernel = sum_{k=1..m} N_k(x)*N_k(y).
    We'll compute N_k(x) and N_k(y) on the fly.
    """
    Nx = evaluate_newton_basis(x, kernel_func, pgreedy_info, Omega_b)
    Ny = evaluate_newton_basis(y, kernel_func, pgreedy_info, Omega_b)
    return np.dot(Nx, Ny)

###########################################################################
# Example usage
###########################################################################



if __name__=="__main__":
    # Suppose we have a discrete set Omega_b in [0,1]^3 (or [0, pi]^3).
    # We'll just create a random set for demonstration.
    np.random.seed(None)
    d=12
    nB=1000
    Omega_b = np.random.rand(nB,d)*2*math.pi

    # We'll run P-greedy to get m=10 basis
    max_m = 10
    pgreedy_info = p_greedy_newton_flexible(full_kernel_classic, Omega_b, max_m)

    # Now pick some "new" points x,y that might not be in Omega_b
    x_test = np.random.rand(d) * 2 * math.pi
    y_test = np.random.rand(d) * 2 * math.pi

    # Evaluate approximate kernel
    k_approx = approximate_kernel_flexible(x_test, y_test, full_kernel_classic, pgreedy_info, Omega_b)

    # Compare to true kernel
    k_true = full_kernel_classic(x_test, y_test)

    print(f"Approx kernel(x_test,y_test) = {k_approx:.5f}")
    print(f"True  kernel(x_test,y_test)  = {k_true:.5f}")


In [ ]:
x_test = np.random.rand(d) * 2 * math.pi
y_test = np.random.rand(d) * 2 * math.pi

# Evaluate approximate kernel
k_approx = approximate_kernel_flexible(x_test, y_test, full_kernel_classic, pgreedy_info, Omega_b)

# Compare to true kernel
k_true = full_kernel_classic(x_test, y_test)

print(f"Approx kernel(x_test,y_test) = {k_approx:.5f}")
print(f"True  kernel(x_test,y_test)  = {k_true:.5f}")

In [ ]:
##############################################################################
# 7) Example Usage
##############################################################################
if __name__=="__main__":
    # Example: pick any partial kernel, e.g. kernel_fn4 => subset_wires=[0,1]
    # Or pick among KERNELS = [kernel_fn1,...,kernel_fn7].
    # We'll do a small run for demonstration.

    dim = 100
    Omega_b = np.random.rand(1000,12)*2*math.pi

    max_m = dim
    pgreedy_info = p_greedy_newton_flexible(full_kernel_classic, Omega_b, max_m)
    approx_kernel = lambda x,y: approximate_kernel_flexible(x,y, full_kernel_classic, pgreedy_info, Omega_b)

    def kernell(X,Y):
        K = np.zeros((X.shape[0],Y.shape[0]))
        for i in range(X.shape[0]):
          for j in range(Y.shape[0]):
            K[i,j] = approx_kernel(X[i],Y[j])
        return K

    X_data, y_data, best_hist = bo_with_lbfgs(
        kernel_fn=kernell,
        n_init=5,
        n_iter=30,
        seed=None,
        n_cand=10
    )
    print("Final best energy found:", best_hist[-1])

    # Plot
    plt.plot(best_hist, label="Best so far")
    plt.xlabel("Iteration")
    plt.ylabel("Energy")
    plt.title("BO with Partial-Density Kernel + LBFGS (subset [0,1])")
    plt.legend()
    plt.show()

    # If you want to loop over all partial kernels, you could do:
    # for i, kfn in enumerate(KERNELS):
    #     Xd, yd, bh = bo_with_lbfgs(kfn, n_init=3, n_iter=15, seed=42)
    #     print(f"Kernel #{i+1}: final best energy = {bh[-1]}")

## repeated experiments

In [ ]:

import numpy as np
from numpy.linalg import matrix_rank, svd

def kernel_rank(K, rtol=1e-12):
    """
    Return the numerical rank of a kernel matrix K.

    Parameters
    ----------
    K : ndarray, shape (n_samples, n_samples)
        Symmetric (or Hermitian) kernel Gram matrix.
    rtol : float
        Relative threshold for considering singular values to be zero.
        Default `rtol=1e-12` ≃ np.finfo(float).eps ** 0.5.

    Returns
    -------
    rank : int
        Numerical rank of K.
    """
    # Option 1 – wrapper around NumPy’s built-in rank
    # (uses SVD under the hood, with its own tolerance)
    builtin_rank = matrix_rank(K, tol=rtol)

    # Option 2 – explicit SVD, lets you tune the threshold yourself
    s = svd(K, compute_uv=False)              # singular values (non-negative)
    thresh = rtol * s.max()                   # same criterion NumPy uses
    manual_rank = int((s > thresh).sum())     # count non-negligible values

    # They should be identical; return either
    assert builtin_rank == manual_rank
    return builtin_rank


In [ ]:
Omega_b = np.random.rand(100,12)*2*math.pi
kernel_rank(kernel_fn7(Omega_b,Omega_b))

In [ ]:
if __name__=="__main__":
    # Example usage: define partial kernel on qubits [0,2]
    # Then do BO

    trials = 30
    energy_final = []
    energy_final_std = []

    dims = np.array([2,4,8,16,32,64])

    for dim in dims:
        energies = []
        for trial in range(trials):
            Omega_b = np.random.rand(1000,12)*2*math.pi

            max_m = dim
            pgreedy_info = p_greedy_newton_flexible(full_kernel_classic, Omega_b, max_m)
            approx_kernel = lambda x,y: approximate_kernel_flexible(x,y, full_kernel_classic, pgreedy_info, Omega_b)

            def kernell(X,Y):
                K = np.zeros((X.shape[0],Y.shape[0]))
                for i in range(X.shape[0]):
                  for j in range(Y.shape[0]):
                    K[i,j] = approx_kernel(X[i],Y[j])
                return K


            X_data, y_data, best_hist = bo_with_lbfgs(
                kernel_fn=kernell,
                n_init=5,
                n_iter=50,
                seed=None,
                n_cand=10
            )
            energies.append(best_hist)
            print(f"dim {dim} completed trial {trial}")
            print("Final best energy found:", best_hist[-1])
        energy_all = np.array(energies)
        avg_energy = energy_all.mean(axis=0)
        std_energy = energy_all.std(axis=0)

        energy_final.append(avg_energy[-1])
        energy_final_std.append(std_energy[-1])



    plt.errorbar(dims, np.array(energy_final), np.array(energy_final_std), ls="-",
                marker='d',
                color="#009E73",
                alpha=1.0,
                capsize=4)